# Clean up orphaned snapshots

In [ ]:
from datetime import datetime, timedelta, timezone
from typing import Any

from tqdm.notebook import tqdm
import kubernetes as k8s

In [27]:
k8s.config.load_kube_config("../kubeconfig.yaml", context="queerkastle-de")

In [28]:
# Get all nodes
nodes = k8s.client.CoreV1Api().list_node()
# Print node names
for node in nodes.items:
    print(node.metadata.name)

queerkastle-de-s0


In [29]:
# Get all VolumeSnapshots in all namespaces
snapshot_api = k8s.client.CustomObjectsApi()
volumesnapshots: dict[str, Any] = snapshot_api.list_cluster_custom_object(
  group="snapshot.storage.k8s.io",
  version="v1",
  plural="volumesnapshots"
)

all_snapshots: list[dict[str, Any]] = volumesnapshots.get("items", [])

now = datetime.now(timezone.utc)
old_unready_snapshots = []

for snap in all_snapshots:
  status: dict[str, Any] = snap.get("status", {})
  ready: dict[str, Any] = status.get("readyToUse", True)
  
  meta: dict[str, Any] = snap.get("metadata", {})
  creation: str = meta["creationTimestamp"]
  
  if not ready:
    created_at = datetime.fromisoformat(creation.replace("Z", "+00:00"))
    if now - created_at > timedelta(days=1):
      old_unready_snapshots.append(snap)

print(f"{len(old_unready_snapshots)} out of {len(all_snapshots)}")

0 out of 285


In [30]:
for snap in tqdm(old_unready_snapshots, desc="Deleting old unready snapshots"):
    name = snap["metadata"]["name"]
    namespace = snap["metadata"]["namespace"]
    # Remove finalizers if present
    if "finalizers" in snap["metadata"]:
        snapshot_api.patch_namespaced_custom_object(
            group="snapshot.storage.k8s.io",
            version="v1",
            namespace=namespace,
            plural="volumesnapshots",
            name=name,
            body={"metadata": {"finalizers": []}},
        )
    snapshot_api.delete_namespaced_custom_object(
        group="snapshot.storage.k8s.io",
        version="v1",
        namespace=namespace,
        plural="volumesnapshots",
        name=name,
        body={},
        grace_period_seconds=0,
    )

Deleting old unready snapshots: 0it [00:00, ?it/s]

In [32]:
# Get all VolumeSnapshotContents
volumesnapshotcontents = snapshot_api.list_cluster_custom_object(
    group="snapshot.storage.k8s.io", version="v1", plural="volumesnapshotcontents"
)
all_contents = volumesnapshotcontents.get("items", [])

# Collect all referenced content names from existing snapshots
referenced_content_names = set()
for snap in volumesnapshots.get("items", []):
    status = snap.get("status", {})
    content_name = status.get("boundVolumeSnapshotContentName")
    if content_name:
        referenced_content_names.add(content_name)

# Find orphaned contents (not referenced by any snapshot)
orphaned_contents = [
    content
    for content in all_contents
    if content["metadata"]["name"] not in referenced_content_names
]

print(f"Found {len(orphaned_contents)} orphaned VolumeSnapshotContents.")

# Delete orphaned contents
for content in tqdm(orphaned_contents, desc="Deleting orphaned contents"):
    name = content["metadata"]["name"]
    # Remove finalizers if present
    if "finalizers" in content["metadata"]:
        snapshot_api.patch_cluster_custom_object(
            group="snapshot.storage.k8s.io",
            version="v1",
            plural="volumesnapshotcontents",
            name=name,
            body={"metadata": {"finalizers": []}},
        )
    try:
        snapshot_api.delete_cluster_custom_object(
            group="snapshot.storage.k8s.io",
            version="v1",
            plural="volumesnapshotcontents",
            name=name,
            body={},
            grace_period_seconds=0,
        )
    except k8s.client.ApiException:
        pass

Found 0 orphaned VolumeSnapshotContents.


Deleting orphaned contents: 0it [00:00, ?it/s]